<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Case-List-Refresh-Notebook" data-toc-modified-id="Case-List-Refresh-Notebook-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Case List Refresh Notebook</a></span><ul class="toc-item"><li><span><a href="#TODO" data-toc-modified-id="TODO-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>TODO</a></span></li></ul></li><li><span><a href="#NEW-SPACE" data-toc-modified-id="NEW-SPACE-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>NEW SPACE</a></span></li><li><span><a href="#Logger-Work" data-toc-modified-id="Logger-Work-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Logger Work</a></span></li></ul></div>

# Case List Refresh Notebook

## TODO

- Move cases to Google Drive
- Chron jobs


Thought process: Have the same outputs for both SC and WM and eventually Albertsons. This should be refreshed at least once a week, hopefully done by Jenkins or another chron tool. It should just run as a script rather than a program, though functionality should be given to do both. 

# NEW SPACE

In [10]:
from __future__ import print_function
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

import pygsheets

import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import time

import os
import os.path
from os import path

from shutil import copy
import re

from config import mysql_credentials
from config import powerpoint_paths

# https://pygsheets.readthedocs.io/en/stable/worksheet.html
def mysql_query(query='DEFAULT', verbose=False):
    import sqlalchemy as sql
    username = mysql_credentials['username']
    password = mysql_credentials['password']
    connection = mysql_credentials['connection']
    database_name = mysql_credentials['database_name']
    connect_string = f'mysql://{username}:{password}@{connection}/{database_name}'
    sql_engine = sql.create_engine(connect_string)
    df = pd.read_sql_query(query, sql_engine)
    if verbose: print(df.shape)
    return df

def worksheet(gsheet_url, network):
    gc = pygsheets.authorize(client_secret='client_secret.json')
    sh = gc.open_by_url(gsheet_url)

    network_dict = {
    "Sam's Club":0,
    'Walmart':1
    }
    
    network_choice = network_dict[network] # Relating network to sheet index
    # network_choice = 2 # testing
    wk = sh[network_choice]
    return wk


# GOOGLE SPREADSHEET DATAFRAME UPDATER
def gsheet_import_df(wk, verbose=False):

    gsheet_df = wk.get_as_df()
    
    if verbose: print(df.shape)
    
    print(f'Successfully returned {network}.')
    
    return gsheet_df

def gsheet_programs(gsheet_df):
    print(f'Imported Gsheet Dataframe has (rows, columns): {gsheet_df.shape}.')
    not_list = gsheet_df['Program'].to_list() + ['0', '0']
    not_tuple = tuple(not_list)
    print('Tupled dataframe programs.')
    return not_tuple

def sc_sql_metrics(not_tuple):
    query = f"""
    SELECT metrics.program, metrics.weeks_post, metrics.start_date, metrics.end_date, 
    metrics.item_number, metrics.item_description, metrics.category, metrics.subcategory,
    metrics.category_num, metrics.subcat_num, metrics.Freeosk_clubs, metrics.control_clubs,
    metrics.transactions, metrics.scans, metrics.audience, metrics.engagement, metrics.conversion,
    metrics.repeat, metrics.Immediate, metrics.`HH_A%%`, metrics.`HH_B%%`, metrics.`HH_C%%`,
    lift.freeosk_lift, lift.control_lift, cfg.merchandising_type, cfg.placement_type
    FROM longitudinal.c_metrics as metrics
    JOIN longitudinal.c_lift as lift ON metrics.Program = lift.program_name
    JOIN longitudinal.c_cfg as cfg on metrics.program = cfg.program_name
    WHERE metrics.Weeks_post = '12W'
    AND metrics.program NOT IN {not_tuple}
    AND metrics.program LIKE '%%_12.zip%%';
    """
    raw_sc_df = mysql_query(query)
    print(f'Number of SC rows to append: {len(raw_sc_df.index)}')
    return raw_sc_df

def sc_formatter(raw_sc_df):
    # Formatting
    print(f'Successfully returned the MySQL query')
    sc_df = raw_sc_df.copy()
    sc_df['Traffic/club'] = sc_df['audience'] / sc_df['Freeosk_clubs']
    sc_df['Scans/club'] = sc_df['scans'] / sc_df['Freeosk_clubs']
    sc_df['Program2'] = sc_df['program'].str.split("_", n=1, expand=True)[0]
    sc_df['Unique Identifier'] = sc_df['Program2'] + sc_df['item_number'].astype(str)
    renamed_columns = {
        'HH_A%': 'A%',
        'HH_B%': 'B%',
        'HH_C%': 'C%'
    }
    sc_df = sc_df.rename(columns=renamed_columns)
    sc_df.loc[:, 'Tags'] = ''
    sc_df.loc[:, 'Notes'] = ''
    sc_df.columns = [column.replace('_', ' ') for column in list(sc_df)]
    sc_df.columns = [column.capitalize() for column in list(sc_df)]
    print(f'Successfully formatted the MySQL query')
    
    # Mapping to food boolean
    food_mapping = pd.read_excel('data/Food_Mapping.xlsx')
    sc_df_2 = pd.merge(sc_df, food_mapping,  how='left', 
                       left_on=['Category num','Subcat num'], 
                       right_on = ['CATEGORY_NUMBER','SUB_CATEGORY_NUMBER'])
    print(f'Successfully joined to Food_Mapping.xslx')
    
    # Final column rearrangement
    rearranged_columns = ['Program', 'Tags', 'Notes', 'Weeks post', 'Start date', 
                      'End date', 'Item number', 'Item description', 'Category', 
                      'Subcategory', 'Category num', 'Subcat num', 'Freeosk clubs', 
                      'Control clubs', 'Transactions', 'Audience', 'Traffic/club', 
                      'Scans', 'Scans/club', 'Engagement', 'Conversion', 'Repeat', 
                      'Immediate', 'A%', 'B%', 'C%', 'Freeosk lift', 'Control lift', 
                      'Merchandising type', 'Placement type', 'Program2', 'Unique identifier', 'Food']
    
    sc_df_3 = sc_df_2[rearranged_columns].reset_index(drop=True)
    
    print(f'SC_Formatter completed! Shape is {sc_df_3.shape}')
    
    return sc_df_3  

def wm_sql_metrics(not_tuple):
    query = f"""SELECT traceable.Program_name, 
    traceable.Weeks_post, traceable.Start_date, traceable.End_date,
    traceable.Item_number, traceable.Placement_Name, traceable.Dept,
    traceable.Sub_Dept, traceable.Dept_Name, traceable.Sub_Dept_Name,
    traceable.Freeosk_stores, traceable.Rest_of_Chain_Stores,
    traceable.Traceable_Audience, traceable.Non_Traceable_Aud,
    traceable.Audience, traceable.Scans, traceable.Total_Conv,
    traceable.Repeat, traceable.A, traceable.B, traceable.C,
    traceable.Households, traceable.`A%%`, traceable.`B%%`, traceable.`C%%`, traceable.`A+B%%`,
    lift.Control_lift, lift.Freeosk_lift
    FROM longitudinal.traceable as traceable
    JOIN longitudinal.lift as lift ON traceable.program_name = lift.Program_name
    WHERE traceable.Weeks_post = '4W'
    AND traceable.program_name LIKE '%%_4.zip'
    AND traceable.program_name NOT IN {not_tuple};"""
    raw_wm_df = mysql_query(query)
    print(f'Number of WM rows to append: {len(raw_wm_df.index)}')
    return raw_wm_df

def wm_formatter(raw_wm_df):
    # Formatting
    print(f'Successfully returned the MySQL query')
    wm_df = raw_wm_df.copy()
    wm_df['Traffic/store'] = wm_df['Audience'] / wm_df['Freeosk_stores']
    wm_df['Scans/store'] = wm_df['Scans'] / wm_df['Freeosk_stores']
    wm_df['Program2'] = wm_df['Program_name'].str.split("_", n=1, expand=True)[0]
    wm_df.loc[:, 'Tags'] = ''
    wm_df.loc[:, 'Notes'] = ''
    wm_df.columns = [column.replace('_', ' ') for column in list(wm_df)]
    wm_df.columns = [column.capitalize() for column in list(wm_df)]
    print(f'Successfully formatted the MySQL query')
    
    # Final column rearrangement
    rearranged_columns = ['Program name', 'Tags', 'Notes', 'Weeks post', 'Start date', 'End date', 
                          'Item number', 'Placement name', 'Dept', 'Sub dept', 'Dept name', 
                          'Sub dept name', 'Freeosk stores', 'Rest of chain stores', 'Traceable audience', 
                          'Non traceable aud', 'Audience', 'Traffic/store', 'Scans', 'Scans/store', 'Total conv', 
                          'Repeat', 'A', 'B', 'C', 'Households', 'A%', 'B%', 'C%', 'A+b%', 'Control lift', 
                          'Freeosk lift', 'Program2']
    
    wm_df = wm_df[rearranged_columns].reset_index(drop=True)
    
    rename_col = {'A+b%':'A+B%', 'Program name': 'Program'}
    wm_df = wm_df.rename(columns=rename_col)
    print(f'WM_Formatter completed! Shape is {wm_df.shape}')
    
    return wm_df

def gsheet_uploader(wk, gsheet_df, append_df):
    gsheet_import_appended = gsheet_df.append(append_df, ignore_index=True)
    
    
    # wk.clear('A2') # A2 is start of dataframe
    # wk.set_dataframe(gsheet_import_appended, 'A2', copy_index=False, copy_head=False, extend=False, fit=True, escape_formulae=True, nan='')
    print('New data has been successfully uploaded!') 


# HYPERLINK FUNCTIONS
def gfile_list_agg():

    # If modifying these scopes, delete the file token.pickle.
    SCOPES = ['https://www.googleapis.com/auth/drive.metadata.readonly']

    """Shows basic usage of the Drive v3 API.
    Prints the names and ids of the first 10 files the user has access to.
    """
    creds = None
    # The file token.pickle stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                '../credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    service = build('drive', 'v3', credentials=creds)

    # Call the Drive v3 API

    page_token = None
    file_list = {}
    page = 0

    while True:
        results = service.files().list(q="'1qQZpAhzmKoR7drBYh8kYWVS6GaMPCxr3' in parents", spaces='drive',
                                              fields='nextPageToken, files(id, name)',
                                              pageToken=page_token).execute()
        items = results.get('files', [])

        for item in items:
            file_list[item['name']] = item['id']

        page_token = results.get('nextPageToken', None)
        if page_token is None: break

    print('Google Drive: PPTX files collected')
    print(f'There are {len(file_list)} PPTX files in the Cases folder.')
    return file_list

def hyperlink_updater(wk, file_list):
    
    n_rows = wk.rows + 1
    for i in range(2, n_rows):
        
        cell = pygsheets.Cell(f'A{i}', worksheet=wk, cell_data=None)
        cell_value = wk.get_value(f'A{i}')
        
        if not cell.formula and cell_value: # Must have value but no hyperlink

            print(f'Cell A{i} requires a hyperlink: {cell_value}')
            
            file_name = cell_value.split('.')[0] + '_case.pptx' # Name to help locate file names
            cell_nbr = f'A{i}'
                        
            file_list_names = list(file_list.keys())
            
            if file_name in file_list_names:
                print(f'Found one {file_name}!')

                file_name_zip = file_name.split('.pptx')[0] + '.zip' # Reference back to original name

                cell.formula = f'=HYPERLINK("https://drive.google.com/file/d/{file_list[file_name]}", "{file_name_zip}")'

                print(f'Replaced {cell_nbr}: {file_name} : {file_list[file_name]}')
                
            else: 
                print(f'COULD NOT REPLACE {cell_nbr}: {file_name}')

In [ ]:
gsheet_url = 'https://docs.google.com/spreadsheets/d/1wsnBd3AHObl4gnUJ2MlwkusuXRoOsQpu2Kx6zGH8bVY/edit#gid=0'
network_name_sub = "Sam's Club"
wk = worksheet(gsheet_url, network_name_sub)
wk.find('', searchByRegex=False, matchCase=False, matchEntireCell=False, includeFormulas=False, cols=None, rows=None, forceFetch=True)

In [108]:
gsheet_url = 'https://docs.google.com/spreadsheets/d/1-p5-secff5mixTwj2QSqd7rl4twcmZBmQBaR1LrM2Pw/edit#gid=0'
# network_name = initial_prompts('Case List Refresher')
network_name = 'All'
file_list = gfile_list_agg()



Google Drive: PPTX files collected
There are 726 PPTX files in the Cases folder.


In [109]:
if network_name == 'All' or network_name == "Sam's Club":
    network_name_sub = "Sam's Club"
    wk = worksheet(gsheet_url, network_name_sub)
    hyperlink_updater(wk, file_list)

Cell A2 requires a hyperlink: FSC14005_475453_20140201_12.zip
Found one FSC14005_475453_20140201_12_case.pptx!
Replaced A2: FSC14005_475453_20140201_12_case.pptx : 1BliWf97MZCRpGG3Oj2OdMr1PKu5edxwS
Cell A3 requires a hyperlink: FSC14029_103575_20140531_12.zip
Found one FSC14029_103575_20140531_12_case.pptx!
Replaced A3: FSC14029_103575_20140531_12_case.pptx : 1pijrtqC_4mqgLE02p9cSyolCBR0JODb5
Cell A4 requires a hyperlink: FSC14033_40783_20140607_12.zip
Found one FSC14033_40783_20140607_12_case.pptx!
Replaced A4: FSC14033_40783_20140607_12_case.pptx : 1RKS4bIv_C6LkCaBiif8T9bEFeNQ56PKH
Cell A5 requires a hyperlink: FSC14098_876310_20150321_12.zip
Found one FSC14098_876310_20150321_12_case.pptx!
Replaced A5: FSC14098_876310_20150321_12_case.pptx : 1kIc2qo-PrNwNkExFubjuSPhofBgz9a1Q
Cell A6 requires a hyperlink: FSC15042_222027_20150509_12.zip
Found one FSC15042_222027_20150509_12_case.pptx!
Replaced A6: FSC15042_222027_20150509_12_case.pptx : 1o4KiRp9jSxCLjtMvf4FF7MxjQ70qgAqY
Cell A7 requi

Replaced A43: FSC16109_347715_20170128_12_case.pptx : 1kRiGdfgD-v_Kw9VMO2ruLw2m-Widzg9p
Cell A44 requires a hyperlink: FSC16109_80643_20170128_12.zip
Found one FSC16109_80643_20170128_12_case.pptx!
Replaced A44: FSC16109_80643_20170128_12_case.pptx : 17-yxw-RGNsD14iqb8oFEdK1wrXPVvPU0
Cell A45 requires a hyperlink: FSC16094_865538_20170204_12.zip
Found one FSC16094_865538_20170204_12_case.pptx!
Replaced A45: FSC16094_865538_20170204_12_case.pptx : 18ZlCX4mz48KSW4h1T-RBcCQXAY2D_Ooo
Cell A46 requires a hyperlink: FSC16096_708449_20170204_12.zip
Found one FSC16096_708449_20170204_12_case.pptx!
Replaced A46: FSC16096_708449_20170204_12_case.pptx : 1F521SMO2nejXYWyU3o4ZqFsIeLHfE5fN
Cell A47 requires a hyperlink: FSC16103_391727_20170204_12.zip
Found one FSC16103_391727_20170204_12_case.pptx!
Replaced A47: FSC16103_391727_20170204_12_case.pptx : 16G4XZnZeBN5oTvBm3e1grA3UeuSRk-Rd
Cell A48 requires a hyperlink: FSC16103_561914_20170204_12.zip
Found one FSC16103_561914_20170204_12_case.pptx!
Rep

Cell A85 requires a hyperlink: FSC17001_334505_20170304_12.zip
Found one FSC17001_334505_20170304_12_case.pptx!
Replaced A85: FSC17001_334505_20170304_12_case.pptx : 1ZqKhuiNa5BvXtEJsh-maMsKsUVcg5q3W
Cell A86 requires a hyperlink: FSC17001_623387_20170304_12.zip
Found one FSC17001_623387_20170304_12_case.pptx!
Replaced A86: FSC17001_623387_20170304_12_case.pptx : 1TY8F9cghB_FNQq9zuNPEr2Wy7kUEx6pI
Cell A87 requires a hyperlink: FSC17010_995681_20170304_12.zip
Found one FSC17010_995681_20170304_12_case.pptx!
Replaced A87: FSC17010_995681_20170304_12_case.pptx : 1F-faFHmLpzvoIJRYKPPpNLqyeIZNs99n
Cell A88 requires a hyperlink: FSC16123_948851_20170311_12.zip
Found one FSC16123_948851_20170311_12_case.pptx!
Replaced A88: FSC16123_948851_20170311_12_case.pptx : 1_0vy8y7sRDO9Wslw857gJLB7EoFI8y1M
Cell A89 requires a hyperlink: FSC15068_965816_20170318_12.zip
Found one FSC15068_965816_20170318_12_case.pptx!
Replaced A89: FSC15068_965816_20170318_12_case.pptx : 1zqCY952POpIuyB3X7zs5L_HoWgfJoRr9


Cell A126 requires a hyperlink: FSC17016_402807_20170429_12.zip
Found one FSC17016_402807_20170429_12_case.pptx!
Replaced A126: FSC17016_402807_20170429_12_case.pptx : 19K-fYILxUxHVmLhYQKj_bFsCV_CYlhS7
Cell A127 requires a hyperlink: FSC17016_852120_20170429_12.zip
Found one FSC17016_852120_20170429_12_case.pptx!
Replaced A127: FSC17016_852120_20170429_12_case.pptx : 1m0UuoPJkrjkvHon9pfVjxh-7AvqgHpPH
Cell A128 requires a hyperlink: FSC17020_769693_20170429_12.zip
Found one FSC17020_769693_20170429_12_case.pptx!
Replaced A128: FSC17020_769693_20170429_12_case.pptx : 1UTM3GeZPoDN1Ahe7c6JcM1ZnjruhFcAt
Cell A129 requires a hyperlink: FSC17020_973909_20170429_12.zip
Found one FSC17020_973909_20170429_12_case.pptx!
Replaced A129: FSC17020_973909_20170429_12_case.pptx : 1KBWpz1ptYA0mpLax0cNejIii8jFFBAvv
Cell A130 requires a hyperlink: FSC17008_980006358_20170506_12.zip
Found one FSC17008_980006358_20170506_12_case.pptx!
Replaced A130: FSC17008_980006358_20170506_12_case.pptx : 1TCGT0EYsIv5s9Q

Replaced A166: FSC17073_951129_20170617_12_case.pptx : 1F8X0_rFdWd7YQHhC1IN_S9kSHxYC2npC
Cell A167 requires a hyperlink: FSC17078_190736_20170624_12.zip
Found one FSC17078_190736_20170624_12_case.pptx!
Replaced A167: FSC17078_190736_20170624_12_case.pptx : 1QbEbgpgi2YhNrLjFQS7k_i8EvN3FNjuj
Cell A168 requires a hyperlink: FSC17078_705387_20170624_12.zip
Found one FSC17078_705387_20170624_12_case.pptx!
Replaced A168: FSC17078_705387_20170624_12_case.pptx : 1jrsjMVanS7rc1Y6MHGljCiyALCU1RGC2
Cell A169 requires a hyperlink: FSC17078_709743_20170624_12.zip
Found one FSC17078_709743_20170624_12_case.pptx!
Replaced A169: FSC17078_709743_20170624_12_case.pptx : 1TYx8AGTHlbpqSdqz0Qa0Ttvqmi5LWiNu
Cell A170 requires a hyperlink: FSC17082_190736_20170701_12.zip
Found one FSC17082_190736_20170701_12_case.pptx!
Replaced A170: FSC17082_190736_20170701_12_case.pptx : 1t8Ua5xySElucdNfOY_xUOwThKpGrUZoj
Cell A171 requires a hyperlink: FSC17082_705387_20170701_12.zip
Found one FSC17082_705387_20170701_12_c

...

Cell A409 requires a hyperlink: FSC18018_705387_20180629_12.zip
COULD NOT REPLACE A409: FSC18018_705387_20180629_12_case.pptx
Cell A410 requires a hyperlink: FSC18018_709743_20180629_12.zip
COULD NOT REPLACE A410: FSC18018_709743_20180629_12_case.pptx
Cell A411 requires a hyperlink: FSC18052_980100185_20180629_12.zip
COULD NOT REPLACE A411: FSC18052_980100185_20180629_12_case.pptx
Cell A412 requires a hyperlink: FSC18056_980087055_20180706_12.zip
COULD NOT REPLACE A412: FSC18056_980087055_20180706_12_case.pptx
Cell A413 requires a hyperlink: FSC18044_358273_20180713_12.zip
COULD NOT REPLACE A413: FSC18044_358273_20180713_12_case.pptx
Cell A414 requires a hyperlink: FSC18044_980058839_20180713_12.zip
COULD NOT REPLACE A414: FSC18044_980058839_20180713_12_case.pptx
Cell A415 requires a hyperlink: FSC18047_567934_20180713_12.zip
COULD NOT REPLACE A415: FSC18047_567934_20180713_12_case.pptx
Cell A416 requires a hyperlink: FSC18047_980058839_20180713_12.zip
COULD NOT REPLACE A416: FSC18047_

Cell A471 requires a hyperlink: FSC18075_57116_20181005_12.zip
Found one FSC18075_57116_20181005_12_case.pptx!
Replaced A471: FSC18075_57116_20181005_12_case.pptx : 1PGSrmUAyBBveJEU6MOP9gkYnMT7NrFxF
Cell A472 requires a hyperlink: FSC18075_57189_20181005_12.zip
Found one FSC18075_57189_20181005_12_case.pptx!
Replaced A472: FSC18075_57189_20181005_12_case.pptx : 1oj90ICIiVdjkfxuPfWBmEY8TmdgKZxJR
Cell A473 requires a hyperlink: FSC18087_980136022_20181005_12.zip
Found one FSC18087_980136022_20181005_12_case.pptx!
Replaced A473: FSC18087_980136022_20181005_12_case.pptx : 1YU-CpjGpyh5lq6WMFnwyxPoVkhOqCt5w
Cell A474 requires a hyperlink: FSC18093_11808_20181005_12.zip
Found one FSC18093_11808_20181005_12_case.pptx!
Replaced A474: FSC18093_11808_20181005_12_case.pptx : 1Ane3SQiB5Br18XnBRTaIc5fSXgs1mPC7
Cell A475 requires a hyperlink: FSC18093_944440_20181005_12.zip
Found one FSC18093_944440_20181005_12_case.pptx!
Replaced A475: FSC18093_944440_20181005_12_case.pptx : 1Dd5m0T3g7pRogKoBG5HZrOj

Cell A511 requires a hyperlink: FSC18096_980070881_20181207_12.zip
Found one FSC18096_980070881_20181207_12_case.pptx!
Replaced A511: FSC18096_980070881_20181207_12_case.pptx : 1MdjRjfb0tBiNB1wFbrYOZVcGHpZuDj4n
Cell A512 requires a hyperlink: FSC18096_980126238_20181207_12.zip
Found one FSC18096_980126238_20181207_12_case.pptx!
Replaced A512: FSC18096_980126238_20181207_12_case.pptx : 1w5fAjyxqdE09mLXziWo_Mj2NuAYJqcNK
Cell A513 requires a hyperlink: FSC18098_980114984_20181207_12.zip
Found one FSC18098_980114984_20181207_12_case.pptx!
Replaced A513: FSC18098_980114984_20181207_12_case.pptx : 12Zub2wC3tYyUEnK2JmLmUp4rMvYUOLJ4
Cell A514 requires a hyperlink: FSC18107_92801_20181207_12.zip
Found one FSC18107_92801_20181207_12_case.pptx!
Replaced A514: FSC18107_92801_20181207_12_case.pptx : 12t5Q4QlZO-44hxoZM10M3S1k6xeZyi9P
Cell A515 requires a hyperlink: FSC18108_326282_20181207_12.zip
Found one FSC18108_326282_20181207_12_case.pptx!
Replaced A515: FSC18108_326282_20181207_12_case.pptx : 

Cell A551 requires a hyperlink: FSC18063_980009095_20190308_12.zip
Found one FSC18063_980009095_20190308_12_case.pptx!
Replaced A551: FSC18063_980009095_20190308_12_case.pptx : 1_5qjcnS4NiE9e_TO71RvKQTZ7aQ8_mek
Cell A552 requires a hyperlink: FSC19004_980141899_20190308_12.zip
Found one FSC19004_980141899_20190308_12_case.pptx!
Replaced A552: FSC19004_980141899_20190308_12_case.pptx : 135qjGSnbXg0iwYc0hloHVeC_J9PMKkpi
Cell A553 requires a hyperlink: FSC19004_980156320_20190308_12.zip
Found one FSC19004_980156320_20190308_12_case.pptx!
Replaced A553: FSC19004_980156320_20190308_12_case.pptx : 1PD6G4VheJY59RgEp_37WX8KE7phoVJTi
Cell A554 requires a hyperlink: FSC19004_980157831_20190308_12.zip
Found one FSC19004_980157831_20190308_12_case.pptx!
Replaced A554: FSC19004_980157831_20190308_12_case.pptx : 1f2egmUWWEMMg5f1v4VOJhGnE2uMuekAV
Cell A555 requires a hyperlink: FSC19006_151618_20190308_12.zip
Found one FSC19006_151618_20190308_12_case.pptx!
Replaced A555: FSC19006_151618_20190308_12_

Replaced A590: FSC19016_580068_20190517_12_case.pptx : 1v91h2h0Jfcg74Y_xpuCaxA-syMYQc_OM
Cell A591 requires a hyperlink: FSC19020_980139499_20190517_12.zip
Found one FSC19020_980139499_20190517_12_case.pptx!
Replaced A591: FSC19020_980139499_20190517_12_case.pptx : 15a3OY4tJ_5VCkXO8Mm1GXt2xSIivvu7X
Cell A592 requires a hyperlink: FSC19024_850799_20190517_12.zip
Found one FSC19024_850799_20190517_12_case.pptx!
Replaced A592: FSC19024_850799_20190517_12_case.pptx : 1_MxFRrUt6lHokHimdOstaPyul9x0P-FL
Cell A593 requires a hyperlink: FSC18018_190736_20190524_12.zip
Found one FSC18018_190736_20190524_12_case.pptx!
Replaced A593: FSC18018_190736_20190524_12_case.pptx : 1RqtQMpjl-GLuj-lPhSOTLacz6cwZZ0xU
Cell A594 requires a hyperlink: FSC18018_705387_20190524_12.zip
Found one FSC18018_705387_20190524_12_case.pptx!
Replaced A594: FSC18018_705387_20190524_12_case.pptx : 11OzKc0aSalQ2QR8STMhTuyy_flAPwlLh
Cell A595 requires a hyperlink: FSC18018_709743_20190524_12.zip
Found one FSC18018_709743_2019

In [110]:
if network_name == 'All' or network_name == "Walmart":
    network_name_sub = 'Walmart'
    wk = worksheet(gsheet_url, network_name_sub)
    hyperlink_updater(wk, file_list)

Cell A2 requires a hyperlink: ACWM190104S104_01_4.zip
COULD NOT REPLACE A2: ACWM190104S104_01_4_case.pptx
Cell A3 requires a hyperlink: ACWM190104S104_02_4.zip
COULD NOT REPLACE A3: ACWM190104S104_02_4_case.pptx
Cell A4 requires a hyperlink: ACWM190111C112_42_4.zip
COULD NOT REPLACE A4: ACWM190111C112_42_4_case.pptx
Cell A5 requires a hyperlink: ACWM190111S106_01_4.zip
Found one ACWM190111S106_01_4_case.pptx!
Replaced A5: ACWM190111S106_01_4_case.pptx : 1mN4bQWguqzgGaXdetCov0ujxzME1FH6c
Cell A6 requires a hyperlink: ACWM190111S106_02_4.zip
Found one ACWM190111S106_02_4_case.pptx!
Replaced A6: ACWM190111S106_02_4_case.pptx : 1jMx7-XP5DlaDrvDxmGUaUmwOjo5F2z1c
Cell A7 requires a hyperlink: ACWM190111S112_01_4.zip
COULD NOT REPLACE A7: ACWM190111S112_01_4_case.pptx
Cell A8 requires a hyperlink: ACWM190111S112_02_4.zip
COULD NOT REPLACE A8: ACWM190111S112_02_4_case.pptx
Cell A9 requires a hyperlink: ACWM190111S115_01_4.zip
COULD NOT REPLACE A9: ACWM190111S115_01_4_case.pptx
Cell A10 require

Cell A72 requires a hyperlink: ACWM190719C157_45_4.zip
COULD NOT REPLACE A72: ACWM190719C157_45_4_case.pptx
Cell A73 requires a hyperlink: ACWM190719C157_46_4.zip
COULD NOT REPLACE A73: ACWM190719C157_46_4_case.pptx
Cell A74 requires a hyperlink: ACWM190719C454_42_4.zip
COULD NOT REPLACE A74: ACWM190719C454_42_4_case.pptx
Cell A75 requires a hyperlink: ACWM190719S154_02_4.zip
COULD NOT REPLACE A75: ACWM190719S154_02_4_case.pptx
Cell A76 requires a hyperlink: ACWM190719S157_02_4.zip
COULD NOT REPLACE A76: ACWM190719S157_02_4_case.pptx
Cell A77 requires a hyperlink: ACWM190726C161_45_4.zip
COULD NOT REPLACE A77: ACWM190726C161_45_4_case.pptx
Cell A78 requires a hyperlink: ACWM190726C161_46_4.zip
Found one ACWM190726C161_46_4_case.pptx!
Replaced A78: ACWM190726C161_46_4_case.pptx : 1eBxsjInK52zYvt1y6xh9eegjkNm0kK_S
Cell A79 requires a hyperlink: ACWM190726C173_42_4.zip
COULD NOT REPLACE A79: ACWM190726C173_42_4_case.pptx
Cell A80 requires a hyperlink: ACWM190726S161_01_4.zip
COULD NOT REP

In [7]:
def ppt_migrator(network):
    
    if network == "Sam's Club": src = powerpoint_paths['sc_src']
    elif network == 'Walmart': src = powerpoint_paths['wm_src']
        
    files = []
    # r=root, d=directories, f = files
    for r, d, f in os.walk(src):
        for file in f:
            if 'case.pptx' in file:
                files.append(os.path.join(r, file))
    if not files: print('No files found, please check source path.')
    
    dst = powerpoint_paths['dst']

    count = 0

    for x in files:
        if network == "Sam's Club": x_split = re.split('(FSC.*)', x)[1]
        elif network == "Walmart": x_split = re.split('(ACWM.*)', x)[1]
            
        if path.exists(dst + '\\' + x_split): pass
        else:
            print(f'Replacing {x_split}.')
            copy(x, dst)
            count += 1

    print(f'{count} files migrated.')

ppt_migrator("Sam's Club")

0 files migrated.


In [24]:
mysql_query("SELECT * from analytics.v_analytics_priority WHERE primary_name LIKE '%%Dove%%'").to_csv('forsteve_dove.csv')

In [20]:
query = """
SELECT *
    FROM longitudinal.traceable as traceable
    JOIN longitudinal.lift as lift ON traceable.program_name = lift.Program_name
    WHERE traceable.Dept_Name LIKE 'HOUSEHOLD CHEMICALS/SUPP'
    ORDER BY lift.Freeosk_lift desc
    """

mysql_query(query)

,id,Program_name,Weeks_post,Start_date,End_date,Item_number,Placement_Name,Dept,Sub_Dept,Dept_Name,Sub_Dept_Name,Freeosk_stores,Rest_of_Chain_Stores,Traceable_Audience,Non_Traceable_Aud,Audience,Scans,Immediate Conv,Wk_1_Conv,Wk_2_Conv,Wk_3_Conv,Wk_4_Conv,Wk_5_Conv,Wk_6_Conv,Wk_7_Conv,Wk_8_Conv,Wk_9_Conv,Wk_10_Conv,Wk_11_Conv,Wk_12_Conv,Delayed_Conv,Total_Conv,Repeat_Conv,Repeat_A_Count,Repeat_B_Count,Repeat_C_Count,Repeat,Repeat_A%,Repeat_B%,Repeat_C%,Immediate,Delayed,PW_A,W1_A,W2_A,W3_A,W4_A,W5_A,W6_A,W7_A,W8_A,W9_A,W10_A,W11_A,W12A,PW_B,W1_B,W2_B,W3_B,W4_B,W5_B,W6_B,W7_B,W8_B,W9_B,W10_B,W11_B,W12_B,PW_C,W1_C,W2_C,W3_C,W4_C,W5_C,W6_C,W7_C,W8_C,W9_C,W10_C,W11_C,W12_C,A,B,C,Households,A%,B%,C%,A+B%,id,Program_name,Start_date,Pre_Period_End,Freeosk_Prior_4,Control_Prior_4,Freeosk_Prior_4_per_week,Control_Prior_4_per_week,Freeosk_lift,Control_lift,Freeosk_stores,Control_stores,F_PW_per_store,F_Prev4_per_store,C_PW_per_store,C_Prev_4_per_store
0,657,ACWM190215S101_02_4.zip,12W,2019-02-15,2019-02-28,"571856529, 571856530, 571885389, 567455054, 57...",WM Air Wick Warmers Coconut,13,1449,HOUSEHOLD CHEMICALS/SUPP,OILS,700,2858,16443844,8939170,25383014,323667,78874,18640,18015,25656,15797,0,0,0,0,0,0,0,0,78108,156982,8255,2448,2516,3291,0.0526,0.2965,0.3048,0.3987,0.5024,0.4976,44489,7193,6811,13435,6263,0,0,0,0,0,0,0,0,21335,6518,6464,7883,5610,0,0,0,0,0,0,0,0,13050,4929,4740,4338,3924,0,0,0,0,0,0,0,0,78191,47810,30981,156982,0.4981,0.3046,0.1974,0.8026,219,ACWM190215S101_02_4.zip,2019-02-15,2019-02-14,655746.09,1675957.41,163936.523,418989.353,0.8960,0.0673,700,2858,444.044,234.195,156.476,146.602
1,656,ACWM190215S101_02_4.zip,4W,2019-02-15,2019-02-28,"571856529, 571856530, 571885389, 567455054, 57...",WM Air Wick Warmers Coconut,13,1449,HOUSEHOLD CHEMICALS/SUPP,OILS,700,2858,16443844,8939170,25383014,323667,78874,18640,18015,25656,15797,0,0,0,0,0,0,0,0,78108,156982,8255,2448,2516,3291,0.0526,0.2965,0.3048,0.3987,0.5024,0.4976,44489,7193,6811,13435,6263,0,0,0,0,0,0,0,0,21335,6518,6464,7883,5610,0,0,0,0,0,0,0,0,13050,4929,4740,4338,3924,0,0,0,0,0,0,0,0,78191,47810,30981,156982,0.4981,0.3046,0.1974,0.8026,219,ACWM190215S101_02_4.zip,2019-02-15,2019-02-14,655746.09,1675957.41,163936.523,418989.353,0.8960,0.0673,700,2858,444.044,234.195,156.476,146.602
2,655,ACWM190215S101_02_4.zip,PW,2019-02-15,2019-02-28,"571856529, 571856530, 571885389, 567455054, 57...",WM Air Wick Warmers Coconut,13,1449,HOUSEHOLD CHEMICALS/SUPP,OILS,700,2858,16443844,8939170,25383014,323667,78874,18640,18015,25656,15797,0,0,0,0,0,0,0,0,0,78874,968,431,278,259,0.0123,0.4452,0.2872,0.2676,1.0000,0.0000,44489,7193,6811,13435,6263,0,0,0,0,0,0,0,0,21335,6518,6464,7883,5610,0,0,0,0,0,0,0,0,13050,4929,4740,4338,3924,0,0,0,0,0,0,0,0,44489,21335,13050,78874,0.5641,0.2705,0.1655,0.8345,219,ACWM190215S101_02_4.zip,2019-02-15,2019-02-14,655746.09,1675957.41,163936.523,418989.353,0.8960,0.0673,700,2858,444.044,234.195,156.476,146.602
3,654,ACWM190215S101_02_0.zip,12W,2019-02-15,2019-02-28,"571856529, 571856530, 571885389, 567455054, 57...",WM Air Wick Warmers Coconut,13,1449,HOUSEHOLD CHEMICALS/SUPP,OILS,700,2858,16043480,9031665,25075145,323667,76893,0,0,0,0,0,0,0,0,0,0,0,0,0,76893,940,416,266,258,0.0122,0.4426,0.2830,0.2745,1.0000,0.0000,43304,0,0,0,0,0,0,0,0,0,0,0,0,20859,0,0,0,0,0,0,0,0,0,0,0,0,12730,0,0,0,0,0,0,0,0,0,0,0,0,43304,20859,12730,76893,0.5632,0.2713,0.1656,0.8344,218,ACWM190215S101_02_0.zip,2019-02-15,2019-02-14,655746.09,1675957.41,163936.523,418989.353,0.8960,0.0673,700,2858,444.044,234.195,156.476,146.602
4,653,ACWM190215S101_02_0.zip,4W,2019-02-15,2019-02-28,"571856529, 571856530, 571885389, 567455054, 57...",WM Air Wick Warmers Coconut,13,1449,HOUSEHOLD CHEMICALS/SUPP,OILS,700,2858,16043480,9031665,25075145,323667,76893,0,0,0,0,0,0,0,0,0,0,0,0,0,76893,940,416,266,258,0.0122,0.4426,0.2830,0.2745,1.0000,0.0000,43304,0,0,0,0,0,0,0,0,0,0,0,0,20859,0,0,0,0,0,0,0,0,0,0,0,0,12730,0,0,0,0,0,0,0,0,0,0,0,0,43304,20859,12730,76893,0.5632,0.2713,0.1656,0.8344,218,AC

# Logger Work

In [14]:
import logging
logging.basicConfig(filename='app.log', filemode='a', format='%(name)s - %(levelname)s - %(message)s')

x = 3

y = print(3 + 'success!')

try:
    y
#     raise
except TypeError as e:
    logging.error('No config file found. Please create one', exc_info=True)
    print('NO')

TypeError: unsupported operand type(s) for +: 'int' and 'str'

In [ ]:


x = 0

if x == 0: logging.warning('No files found.')
logging.error('This will get logged to a file')

In [43]:
from __future__ import print_function
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

import pygsheets

import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import time

import os
import os.path
from os import path

from shutil import copy
import re

import logging
logging.basicConfig(filename='app.log', filemode='a', format='%(asctime)s - %(levelname)s - %(message)s', datefmt='%d-%b-%y %H:%M:%S')

try: 
    from config import mysql_credentials
    from config import powerpoint_paths
except ImportError:
    logging.error('No config file found. Please create one', exc_info=True)

# https://pygsheets.readthedocs.io/en/stable/worksheet.html
def mysql_query(query='DEFAULT', verbose=False):
    import sqlalchemy as sql
    username = mysql_credentials['username']
    password = mysql_credentials['password']
    connection = mysql_credentials['connection']
    database_name = mysql_credentials['database_name']
    connect_string = f'mysql://{username}:{password}@{connection}/{database_name}'
    sql_engine = sql.create_engine(connect_string)
    df = pd.read_sql_query(query, sql_engine)
    if verbose: print(df.shape)
    return df

def worksheet(gsheet_url, network):
    gc = pygsheets.authorize(client_secret='../credentials/client_secret.json')
    sh = gc.open_by_url(gsheet_url)

    network_dict = {
    "Sam's Club":0,
    'Walmart':1
    }
    
    network_choice = network_dict[network] # Relating network to sheet index
    # network_choice = 2 # testing
    wk = sh[network_choice]
    return wk


# GOOGLE SPREADSHEET DATAFRAME UPDATER
def gsheet_import_df(wk, verbose=False):

    gsheet_df = wk.get_as_df()
    
    if verbose: print(df.shape)
    
    print(f'Successfully returned dataframe.')
    
    return gsheet_df

def gsheet_programs(gsheet_df):
    print(f'Imported Gsheet Dataframe has (rows, columns): {gsheet_df.shape}.')
    not_list = gsheet_df['Program'].to_list() + ['0', '0']
    not_tuple = tuple(not_list)
    print('Tupled dataframe programs.')
    return not_tuple

def sc_sql_metrics(not_tuple):
    query = f"""
    SELECT metrics.program, metrics.weeks_post, metrics.start_date, metrics.end_date, 
    metrics.item_number, metrics.item_description, metrics.category, metrics.subcategory,
    metrics.category_num, metrics.subcat_num, metrics.Freeosk_clubs, metrics.control_clubs,
    metrics.transactions, metrics.scans, metrics.audience, metrics.engagement, metrics.conversion,
    metrics.repeat, metrics.Immediate, metrics.`HH_A%%`, metrics.`HH_B%%`, metrics.`HH_C%%`,
    lift.freeosk_lift, lift.control_lift, cfg.merchandising_type, cfg.placement_type
    FROM longitudinal.c_metrics as metrics
    JOIN longitudinal.c_lift as lift ON metrics.Program = lift.program_name
    JOIN longitudinal.c_cfg as cfg on metrics.program = cfg.program_name
    WHERE metrics.Weeks_post = '12W'
    AND metrics.program NOT IN {not_tuple}
    AND metrics.program LIKE '%%_12.zip%%';
    """
    raw_sc_df = mysql_query(query)
    print(f'Number of SC rows to append: {len(raw_sc_df.index)}')
    return raw_sc_df

def sc_formatter(raw_sc_df):
    # Formatting
    print(f'Successfully returned the MySQL query')
    sc_df = raw_sc_df.copy()
    sc_df['Traffic/club'] = sc_df['audience'] / sc_df['Freeosk_clubs']
    sc_df['Scans/club'] = sc_df['scans'] / sc_df['Freeosk_clubs']
    sc_df['Program2'] = sc_df['program'].str.split("_", n=1, expand=True)[0]
    sc_df['Unique Identifier'] = sc_df['Program2'] + sc_df['item_number'].astype(str)
    renamed_columns = {
        'HH_A%': 'A%',
        'HH_B%': 'B%',
        'HH_C%': 'C%'
    }
    sc_df = sc_df.rename(columns=renamed_columns)
    sc_df.loc[:, 'Tags'] = ''
    sc_df.loc[:, 'Notes'] = ''
    sc_df.columns = [column.replace('_', ' ') for column in list(sc_df)]
    sc_df.columns = [column.capitalize() for column in list(sc_df)]
    print(f'Successfully formatted the MySQL query')
    
    # Mapping to food boolean
    food_mapping = pd.read_excel('data/Food_Mapping.xlsx')
    sc_df_2 = pd.merge(sc_df, food_mapping,  how='left', 
                       left_on=['Category num','Subcat num'], 
                       right_on = ['CATEGORY_NUMBER','SUB_CATEGORY_NUMBER'])
    print(f'Successfully joined to Food_Mapping.xslx')
    
    # Final column rearrangement
    rearranged_columns = ['Program', 'Tags', 'Notes', 'Weeks post', 'Start date', 
                      'End date', 'Item number', 'Item description', 'Category', 
                      'Subcategory', 'Category num', 'Subcat num', 'Freeosk clubs', 
                      'Control clubs', 'Transactions', 'Audience', 'Traffic/club', 
                      'Scans', 'Scans/club', 'Engagement', 'Conversion', 'Repeat', 
                      'Immediate', 'A%', 'B%', 'C%', 'Freeosk lift', 'Control lift', 
                      'Merchandising type', 'Placement type', 'Program2', 'Unique identifier', 'Food']
    
    sc_df_3 = sc_df_2[rearranged_columns].reset_index(drop=True)
    
    print(f'SC_Formatter completed! Shape is {sc_df_3.shape}')
    
    return sc_df_3  

def wm_sql_metrics(not_tuple):
    query = f"""SELECT traceable.Program_name, 
    traceable.Weeks_post, traceable.Start_date, traceable.End_date,
    traceable.Item_number, traceable.Placement_Name, traceable.Dept,
    traceable.Sub_Dept, traceable.Dept_Name, traceable.Sub_Dept_Name,
    traceable.Freeosk_stores, traceable.Rest_of_Chain_Stores,
    traceable.Traceable_Audience, traceable.Non_Traceable_Aud,
    traceable.Audience, traceable.Scans, traceable.Total_Conv,
    traceable.Repeat, traceable.A, traceable.B, traceable.C,
    traceable.Households, traceable.`A%%`, traceable.`B%%`, traceable.`C%%`, traceable.`A+B%%`,
    lift.Control_lift, lift.Freeosk_lift
    FROM longitudinal.traceable as traceable
    JOIN longitudinal.lift as lift ON traceable.program_name = lift.Program_name
    WHERE traceable.Weeks_post = '4W'
    AND traceable.program_name LIKE '%%_4.zip'
    AND traceable.program_name NOT IN {not_tuple};"""
    raw_wm_df = mysql_query(query)
    print(f'Number of WM rows to append: {len(raw_wm_df.index)}')
    return raw_wm_df

def wm_formatter(raw_wm_df):
    # Formatting
    print(f'Successfully returned the MySQL query')
    wm_df = raw_wm_df.copy()
    wm_df['Traffic/store'] = wm_df['Audience'] / wm_df['Freeosk_stores']
    wm_df['Scans/store'] = wm_df['Scans'] / wm_df['Freeosk_stores']
    wm_df['Program2'] = wm_df['Program_name'].str.split("_", n=1, expand=True)[0]
    wm_df.loc[:, 'Tags'] = ''
    wm_df.loc[:, 'Notes'] = ''
    wm_df.columns = [column.replace('_', ' ') for column in list(wm_df)]
    wm_df.columns = [column.capitalize() for column in list(wm_df)]
    print(f'Successfully formatted the MySQL query')
    
    # Final column rearrangement
    rearranged_columns = ['Program name', 'Tags', 'Notes', 'Weeks post', 'Start date', 'End date', 
                          'Item number', 'Placement name', 'Dept', 'Sub dept', 'Dept name', 
                          'Sub dept name', 'Freeosk stores', 'Rest of chain stores', 'Traceable audience', 
                          'Non traceable aud', 'Audience', 'Traffic/store', 'Scans', 'Scans/store', 'Total conv', 
                          'Repeat', 'A', 'B', 'C', 'Households', 'A%', 'B%', 'C%', 'A+b%', 'Control lift', 
                          'Freeosk lift', 'Program2']
    
    wm_df = wm_df[rearranged_columns].reset_index(drop=True)
    
    rename_col = {'A+b%':'A+B%', 'Program name': 'Program'}
    wm_df = wm_df.rename(columns=rename_col)
    print(f'WM_Formatter completed! Shape is {wm_df.shape}')
    
    return wm_df

def gsheet_uploader(wk, gsheet_df, append_df):
    gsheet_import_appended = gsheet_df.append(append_df, ignore_index=True)
    
    wk.clear('A2') # A2 is start of dataframe
    wk.set_dataframe(gsheet_import_appended, 'A2', copy_index=False, copy_head=False, extend=False, fit=True, escape_formulae=True, nan='')
    print('New data has been successfully uploaded!') 


# HYPERLINK FUNCTIONS
def gfile_list_agg():

    # If modifying these scopes, delete the file token.pickle.
    SCOPES = ['https://www.googleapis.com/auth/drive.metadata.readonly']

    """Shows basic usage of the Drive v3 API.
    Prints the names and ids of the first 10 files the user has access to.
    """
    creds = None
    # The file token.pickle stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('../credentials/token.pickle'):
        with open('../credentials/token.pickle', 'rb') as token:
            creds = pickle.load(token)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                '../credentials/credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('../credentials/token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    service = build('drive', 'v3', credentials=creds)

    # Call the Drive v3 API

    page_token = None
    file_list = {}
    page = 0

    while True:
        results = service.files().list(q="'1qQZpAhzmKoR7drBYh8kYWVS6GaMPCxr3' in parents", spaces='drive',
                                              fields='nextPageToken, files(id, name)',
                                              pageToken=page_token).execute()
        items = results.get('files', [])

        for item in items:
            file_list[item['name']] = item['id']

        page_token = results.get('nextPageToken', None)
        if page_token is None: break

    print('Google Drive: PPTX files collected')
    print(f'There are {len(file_list)} PPTX files in the Cases folder.')
    return file_list

def hyperlink_updater(wk, file_list):
    
    n_rows = wk.rows + 1
    for i in range(2, n_rows):
        
        cell = pygsheets.Cell(f'A{i}', worksheet=wk, cell_data=None)
        cell_value = wk.get_value(f'A{i}')

        if not cell.formula and cell_value: # Must have value but no hyperlink
            print(f'Cell A{i} requires a hyperlink: {cell_value}')
            
            file_name = cell_value.split('.')[0] + '.pptx' # Name to help locate file names
            cell_nbr = f'A{i}'
                        
            file_list_names = list(file_list.keys())
            
            if file_name in file_list_names:
                print(f'Found one {file_name}!')

                file_name_zip = file_name.split('.pptx')[0] + '.zip' # Reference back to original name

                cell.formula = f'=HYPERLINK("https://drive.google.com/file/d/{file_list[file_name]}", "{file_name_zip}")'

                print(f'Replaced {cell_nbr}: {file_name} : {file_list[file_name]}')
                
            else: 
                print(f'COULD NOT REPLACE {cell_nbr}: {file_name}')

def ppt_migrator(network):
    
    if network == "Sam's Club": src = powerpoint_paths['sc_src']
    elif network == 'Walmart': src = powerpoint_paths['wm_src']
        
    files = []
    # r=root, d=directories, f = files
    for r, d, f in os.walk(src):
        for file in f:
            if 'case.pptx' in file:
                files.append(os.path.join(r, file))
    if not files: print('No files found, please check source path.')
    
    dst = powerpoint_paths['dst']

    count = 0

    for x in files:
        if network == "Sam's Club": x_split = re.split('(FSC.*)', x)[1]
        elif network == "Walmart": x_split = re.split('(ACWM.*)', x)[1]
            
        if path.exists(dst + '\\' + x_split): pass
        else:
            print(f'Replacing {x_split}.')
            copy(x, dst)
            count += 1

    print(f'{count} files migrated.')


In [49]:
query = """ select * from analytics.v_analytics_priority where placement_name like '%%tabasco%%' order by approved_start_date
"""

mysql_query(query)

,item_number,item_name,approved_start_date,approved_end_date,program_eligibility,campaign_code,placement_code,program_code,merchandising_type,primary_nbr,primary_name,network_name,placement_type,instant_savings,last_pull,salesforce_url,project_url,placement_name,kiosk_count,front_render_url
0,3519,TABASCO Pepper Sauce 12oz,2018-09-21,2018-09-27,Completed,CMP18116,PLSC180146,FSC18077,Tier 1 - Merchandised,980011652,"Keebler Club Crackers Snack Stacks, 24ct",Sam's Club USA,Sample + TIS\t,Yes,12.0,https://freeosk.lightning.force.com/lightning/...,https://basecamp.com/2264437/projects/15624072,Club Crackers & Tabasco + Pub Cheese,244,None
1,980115734,Presidents Pub Cheese Cheddar & Bacon,2018-09-21,2018-09-27,12 Week,CMP18116,PLSC180146,FSC18077,Tier 1 - Other,980011652,"Keebler Club Crackers Snack Stacks, 24ct",Sam's Club USA,Sample + TIS\t,Yes,NaN,https://freeosk.lightning.force.com/lightning/...,https://basecamp.com/2264437/projects/15624072,Club Crackers & Tabasco + Pub Cheese,244,None
2,980011652,"Keebler Club Crackers Snack Stacks, 24ct",2018-09-21,2018-09-27,12 Week,CMP18116,PLSC180146,FSC18077,Tier 1 - Sampled,980011652,"Keebler Club Crackers Snack Stacks, 24ct",Sam's Club USA,Sample + TIS\t,Yes,NaN,https://freeosk.lightning.force.com/lightning/...,https://basecamp.com/2264437/projects/15624072,Club Crackers & Tabasco + Pub Cheese,244,None
3,80643,Cheez-Its Original 3 lbs,2018-09-21,2018-09-27,12 Week,CMP18116,PLSC180146,FSC18077,Tier 2 - Merchandised,980011652,"Keebler Club Crackers Snack Stacks, 24ct",Sam's Club USA,Sample + TIS\t,Yes,NaN,https://freeosk.lightning.force.com/lightning/...,https://basecamp.com/2264437/projects/15624072,Club Crackers & Tabasco + Pub Cheese,244,None
4,105119,Keebler Toasteds Party Pack Cracker Assortment...,2018-09-21,2018-09-27,12 Week,CMP18116,PLSC180146,FSC18077,Tier 2 - Merchandised,980011652,"Keebler Club Crackers Snack Stacks, 24ct",Sam's Club USA,Sample + TIS\t,Yes,NaN,https://freeosk.lightning.force.com/lightning/...,https://basecamp.com/2264437/projects/15624072,Club Crackers & Tabasco + Pub Cheese,244,None
5,980059867,Presidents Pub Cheese Cheddar & Sriracha,2018-09-21,2018-09-27,12 Week,CMP18116,PLSC180146,FSC18077,Tier 1 - Other,980011652,"Keebler Club Crackers Snack Stacks, 24ct",Sam's Club USA,Sample + TIS\t,Yes,NaN,https://freeosk.lightning.force.com/lightning/...,https://basecamp.com/2264437/projects/15624072,Club Crackers & Tabasco + Pub Cheese,244,None
6,980115740,Presidents Pub Cheese Cheddar & Jalapeno,2018-09-21,2018-09-27,12 Week,CMP18116,PLSC180146,FSC18077,Tier 1 - Other,980011652,"Keebler Club Crackers Snack Stacks, 24ct",Sam's Club USA,Sample + TIS\t,Yes,NaN,https://freeosk.lightning.force.com/lightning/...,https://basecamp.com/2264437/projects/15624072,Club Crackers & Tabasco + Pub Cheese,244,None
7,17818,TABASCO Brand Original Flavor Hot Sauce (2 fl....,2018-09-21,2018-09-27,12 Week,CMP18116,PLSC180146,FSC18077,Tier 2 - Merchandised,980011652,"Keebler Club Crackers Snack Stacks, 24ct",Sam's Club USA,Sample + TIS\t,Yes,NaN,https://freeosk.lightning.force.com/lightning/...,https://basecamp.com/2264437/projects/15624072,Club Crackers & Tabasco + Pub Cheese,244,None
8,87523,"Famous Amos Chocolate Chip Cookies, 2oz 42pk",2018-09-21,2018-09-27,12 Week,CMP18116,PLSC180146,FSC18077,Tier 2 - Merchandised,980011652,"Keebler Club Crackers Snack Stacks, 24ct",Sam's Club USA,Sample + TIS\t,Yes,NaN,https://freeosk.lightning.force.com/lightning/...,https://basecamp.com/2264437/projects/15624072,Club Crackers & Tabasco + Pub Cheese,244,None
9,570621332,"Great Value Thick & Absorbent Paper Napkins, 3...",2018-11-09,2018-11-15,12 Week,CMP18144,PLWM180130,FWM18087,Tier 2 - Merchandised,9244065,TABASCO Original Red 5oz,Walmart,Sample,No,NaN,https://freeosk.lightning.force.com/lightning/...,https://basecamp.com/2264437/projects/15653285,Tabasco UX B (With WM App Signs),11,None


In [44]:
hyperlink_updater(wk, file_list)


HELLO
HELLO
HELLO
HELLO
HELLO


KeyboardInterrupt: 

In [42]:
gsheet_url = 'https://docs.google.com/spreadsheets/d/1wsnBd3AHObl4gnUJ2MlwkusuXRoOsQpu2Kx6zGH8bVY/edit#gid=0'
network_name_sub = "Sam's Club"
file_list = gfile_list_agg()

wk = worksheet(gsheet_url, network_name_sub)

# gsheet_df = gsheet_import_df(wk)
# not_tuple = gsheet_programs(gsheet_df) # Checks for rows to append
# raw_sc_df = sc_sql_metrics(not_tuple) # Returns rows to append
# if raw_sc_df.empty: 
#     print('No new SC rows to append. Stopping upload.')
#     logging.warning('No new SC rows to append. Stopping upload.')
# else: 
#     sc_df = sc_formatter(raw_sc_df)
#     gsheet_uploader(wk, gsheet_df, sc_df)

# print('----------------------------------------')
# print("Running PPTX migrator for Sam's Club.")
# try: ppt_migrator(network_name_sub)
# except: 
#     logging.error("ppt_migrator failed.", exc_info=True)
#     print('PPTX migrator failed.')

# print('----------------------------------------')
# print("Running hyperlink updater for Sam's Club.")
# except: 
#     logging.error("Sam's Club hyperlink_updater failed.", exc_info=True)
#     print('Hyperlink_updater failed.')
# logging.warning('No new SC rows to append. Stopping upload.')

Google Drive: PPTX files collected
There are 732 PPTX files in the Cases folder.


KeyboardInterrupt: 

In [ ]:
hyperlink_updater()

In [35]:
ppt_migrator("Sam's Club")

HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY
HEY


KeyboardInterrupt: 

In [29]:
src = powerpoint_paths['sc_src']
# print(powerpoint_paths['wm_src'])
src = 'F:\Analytics\Longitudinals - Composite and Assembled'

In [30]:
src

'F:\\Analytics\\Longitudinals - Composite and Assembled'

In [32]:
files = []
for r, d, f in os.walk(src):
    for file in f:
        if 'case.pptx' in file:
            files.append(os.path.join(r, file))
            print('FOUIND ONE')
if not files: print('No files found, please check source path.')

FOUIND ONE
FOUIND ONE
FOUIND ONE
FOUIND ONE
FOUIND ONE
FOUIND ONE
FOUIND ONE
FOUIND ONE
FOUIND ONE
FOUIND ONE
FOUIND ONE
FOUIND ONE
FOUIND ONE
FOUIND ONE
FOUIND ONE
FOUIND ONE
FOUIND ONE
FOUIND ONE
FOUIND ONE
FOUIND ONE
FOUIND ONE
FOUIND ONE
FOUIND ONE
FOUIND ONE
FOUIND ONE
FOUIND ONE
FOUIND ONE
FOUIND ONE
FOUIND ONE
FOUIND ONE
FOUIND ONE
FOUIND ONE
FOUIND ONE
FOUIND ONE
FOUIND ONE
FOUIND ONE
FOUIND ONE
FOUIND ONE
FOUIND ONE
FOUIND ONE
FOUIND ONE
FOUIND ONE
FOUIND ONE
FOUIND ONE
FOUIND ONE
FOUIND ONE
FOUIND ONE
FOUIND ONE
FOUIND ONE
FOUIND ONE
FOUIND ONE
FOUIND ONE
FOUIND ONE
FOUIND ONE
FOUIND ONE
FOUIND ONE
FOUIND ONE
FOUIND ONE
FOUIND ONE
FOUIND ONE
FOUIND ONE
FOUIND ONE
FOUIND ONE
FOUIND ONE
FOUIND ONE
FOUIND ONE
FOUIND ONE
FOUIND ONE
FOUIND ONE
FOUIND ONE
FOUIND ONE
FOUIND ONE
FOUIND ONE
FOUIND ONE
FOUIND ONE
FOUIND ONE
FOUIND ONE
FOUIND ONE
FOUIND ONE
FOUIND ONE
FOUIND ONE
FOUIND ONE
FOUIND ONE
FOUIND ONE
FOUIND ONE
FOUIND ONE
FOUIND ONE
FOUIND ONE
FOUIND ONE
FOUIND ONE
FOUIND ONE

KeyboardInterrupt: 